In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [1]:
import json
import torch

from torch.utils.data.dataloader import DataLoader
from transformers import BertForTokenClassification, AdamW, BertTokenizer, BertTokenizerFast, BatchEncoding, TrainingArguments, Trainer
from dataclasses import dataclass
from typing import List

from ner_pipeline.containers import TraingingBatch
from ner_pipeline.dataset_ner import TrainingDataset
from ner_pipeline.labelset import LabelSet
from ner_pipeline.prepare_data import prepare_data_from_txt

In [2]:
pattern = r' (\b[A-Z][a-z]+\b)(\s\b[A-Z][a-z]+\b)*'
iliad_ner_dataset = prepare_data_from_txt("example-texts/iliad.txt", pattern)

In [3]:
# # Check out the structure of dataset 
# print(len(iliad_ner_dataset))
# print(iliad_ner_dataset[169])

19491
{'content': 'Skill’d in each art, and crown’d with every grace;', 'annotations': []}


In [7]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')
label_set = LabelSet(labels=["CLEntity"]) #Only one label in this dataset

In [8]:
iliad_ner_trainingData = TrainingDataset(
    data=iliad_ner_dataset, tokenizer=tokenizer, label_set=label_set, tokens_per_batch=16
)

In [9]:
dataset_train = iliad_ner_trainingData[:len(iliad_ner_trainingData) // 2]
dataset_eval = iliad_ner_trainingData[len(iliad_ner_trainingData) // 2:]

In [10]:
model = BertForTokenClassification.from_pretrained(
    "bert-base-cased", num_labels=len(iliad_ner_trainingData.label_set.ids_to_label.values())
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [11]:
training_args = TrainingArguments("test_trainer")

In [12]:
trainer = Trainer(
    model=model, args=training_args, train_dataset=dataset_train, eval_dataset=dataset_eval
)

In [13]:
trainer.train()

***** Running training *****
  Num examples = 10030
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3762


KeyboardInterrupt: 

In [ ]:
trainer.save_model('bert_ner_finetuned_iliad-with-gpu-pattern2.model')